In [127]:
import pandas as pd
import numpy as np

In [128]:
import statsmodels
import scipy

import matplotlib.pyplot as plt
%pylab inline

import warnings
warnings.filterwarnings('ignore')

from statsmodels.stats.weightstats import *

Populating the interactive namespace from numpy and matplotlib


In [129]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [130]:
def write_answer(task, n_diff):
    with open(str(task)+".txt", "w") as fout:
        fout.write(str(n_diff))

Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)

для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [131]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [132]:
data.drop('Patient_id', inplace=True, axis=1)

In [133]:
columns = data.columns.values

In [134]:
data.shape

(72, 15749)

In [135]:
data.head()

,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


Проверка на нормальность

In [136]:
norm_prob = 0
for column in columns[1:]:
    if stats.shapiro(data[column]).pvalue > 0.05:
        norm_prob+= 1
print("Нормально распределено: %f элементов" % norm_prob)

Нормально распределено: 7587.000000 элементов


Критерий Стьюдента для генов в разных группах пациентов

In [137]:
normal_data = data[data.Diagnosis=='normal']
neoplasia_data = data[data.Diagnosis=='early neoplasia']
cancer_data = data[data.Diagnosis=='cancer']

In [138]:
normal_neoplasia_pvalue = []
for column in columns[1:]:
        normal_neoplasia_pvalue.append(scipy.stats.ttest_ind(normal_data[column],neoplasia_data[column], equal_var=False)[1])
print("Средние не равны на %f элементов" % sum([1 if item < 0.05 else 0 for item in normal_neoplasia_pvalue]))

Средние не равны на 1575.000000 элементов


In [139]:
cancer_neoplasia_pvalue = []
for column in columns[1:]:
    cancer_neoplasia_pvalue.append(scipy.stats.ttest_ind(cancer_data[column],neoplasia_data[column], equal_var=False)[1])
print("Средние не равны на %f элементов" % sum([1 if item < 0.05 else 0 for item in cancer_neoplasia_pvalue]))

Средние не равны на 3490.000000 элементов


In [140]:
write_answer('task1', sum([1 if item < 0.05 else 0 for item in normal_neoplasia_pvalue]))
write_answer('task2', sum([1 if item < 0.05 else 0 for item in cancer_neoplasia_pvalue]))

#### Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$F_c(C, T) = \frac{T}C, T>C и \frac{C}T, T<C$$


где
C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [141]:
normal_mean_genes = normal_data.mean()
neoplasia_mean_genes = neoplasia_data.mean()
cancer_mean_genes = cancer_data.mean()

Часть 2: поправка методом Холма-Бонферрони

In [142]:
import statsmodels.stats.multitest as smm

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем

1.5.

Обратите внимание, что

* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.

* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [143]:
p_corrected_n_n = smm.multipletests(normal_neoplasia_pvalue,
                                            alpha = 0.05/2,
                                            method = 'holm')[1]

In [144]:
p_corrected_c_n = smm.multipletests(cancer_neoplasia_pvalue,
                                            alpha = 0.05/2,
                                            method = 'holm')[1]

Практическая значимость изменения на статистически значимых изменениях

In [145]:
significant_changes = 0
for i in range(len(p_corrected_n_n)):
    if p_corrected_n_n[i] < 0.05/2:
        if neoplasia_mean_genes[i] >= normal_mean_genes[i]:
            fold_change = neoplasia_mean_genes[i]/normal_mean_genes[i]
        else:
            fold_change = -normal_mean_genes[i]/neoplasia_mean_genes[i]
        if fold_change > 1.5:
            significant_changes += 1

In [146]:
write_answer('task3', significant_changes)

In [147]:
significant_changes = 0
for i in range(len(p_corrected_c_n)):
    if p_corrected_c_n[i] < 0.05/2:
        if cancer_mean_genes[i] >= neoplasia_mean_genes[i]:
            fold_change = cancer_mean_genes[i]/neoplasia_mean_genes[i]
        else:
            fold_change = abs(-neoplasia_mean_genes[i]/cancer_mean_genes[i])
        if fold_change > 1.5:
            significant_changes += 1

In [148]:
write_answer('task4', significant_changes)

Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от

H
0
​	 , когда они есть, и будут чаще отклонять

H
0
​	 , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [149]:
p_corrected_n_n = smm.multipletests(normal_neoplasia_pvalue,
                                            alpha = 0.05/2,
                                            method = 'fdr_bh')[1]

In [150]:
p_corrected_c_n = smm.multipletests(cancer_neoplasia_pvalue,
                                            alpha = 0.05/2,
                                            method = 'fdr_bh')[1]

Практическая значимость изменения на статистически значимых изменениях

In [151]:
significant_changes = 0
for i in range(len(p_corrected_n_n)):
    if p_corrected_n_n[i] < 0.05/2:
        if neoplasia_mean_genes[i] >= normal_mean_genes[i]:
            fold_change = neoplasia_mean_genes[i]/normal_mean_genes[i]
        else:
            fold_change = -normal_mean_genes[i]/neoplasia_mean_genes[i]
        if fold_change > 1.5:
            significant_changes += 1

In [152]:
write_answer('task5', significant_changes)

In [153]:
significant_changes = 0
for i in range(len(p_corrected_c_n)):
    if p_corrected_c_n[i] < 0.05/2:
        if cancer_mean_genes[i] >= neoplasia_mean_genes[i]:
            fold_change = cancer_mean_genes[i]/neoplasia_mean_genes[i]
        else:
            fold_change = abs(-neoplasia_mean_genes[i]/cancer_mean_genes[i])
        if fold_change > 1.5:
            significant_changes += 1

In [154]:
write_answer('task6', significant_changes)